In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy
import re
import string
from collections import Counter

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [38]:
imdb = pd.read_csv('imdb_labelled.txt', sep='\t|', engine='python', header=None)
imdb.columns = ['desc', 'score']

C:\Users\Ryan\Anaconda3\lib\site-packages\pandas\io\parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
C:\Users\Ryan\Anaconda3\lib\site-packages\pandas\io\parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [13]:
with open('imdb_labelled.txt') as imdb_file: # almost always use the 'with' statement 
    text = imdb_file.readlines()    # when opening; dont have to use .close()
    print("This file is {} lines long\n".format(len(text)))
    for line in text:
        print(line)

This file is 1000 lines long

A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  	0

Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  	0

Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  	0

Very little music or anything to speak of.  	0

The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  	1

The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty.  	0

Wasted two hours.  	0

Saw the movie today and thought it was a good effort, good messages for kids.  	1

A bit predictable.  	0

Loved the casting of Jimmy Buffet as the science teacher.  	1

And those baby owls were adorable.  	1

The movie showed a lot of Florida at it's best, made it look very appeali

## decide what features to add, or what words to add to features

In [40]:
imdb['categ'] = imdb['score'].apply(lambda x:'pos' if x==1 else 'neg')

In [42]:
imdb_neg = imdb.copy()[imdb['score'] == 0]
imdb_pos = imdb.copy()[imdb['score'] == 1]


In [43]:
# imdb_pos['score'] = 'pos'
# imdb_neg['score'] = 'neg'

In [44]:
pos_words = imdb_pos['desc'].tolist()

In [45]:
neg_words = imdb_neg['desc'].tolist()

In [119]:
all_words = imdb['desc'].tolist()

In [46]:
def features(sentence):
    words = sentence.lower().split()
    return dict(('%s' % w, True) for w in words)

In [47]:
pos_featuresets = list(map(features, pos_words))

In [48]:
neg_featuresets = list(map(features, neg_words))

In [120]:
text_mnb = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB())
                    ])

In [124]:
X_train, X_test, y_train, y_test = train_test_split(imdb.desc, imdb.score)

In [125]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [126]:
predicted = text_mnb.predict(X_test)

In [127]:
np.mean(predicted == y_test)

0.792

In [136]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.76      0.84      0.80       123
        pos       0.83      0.75      0.79       127

avg / total       0.80      0.79      0.79       250



In [137]:
metrics.confusion_matrix(y_test, predicted)

array([[103,  20],
       [ 32,  95]], dtype=int64)

In [142]:
count_vect = CountVectorizer()

In [143]:
X_train_counts = count_vect.fit_transform(X_train)

In [144]:
count_vect.vocabulary_

{'ll': 1304,
 'even': 735,
 'say': 1869,
 'it': 1177,
 'again': 72,
 'this': 2214,
 'is': 1172,
 'torture': 2259,
 'especially': 728,
 'liked': 1287,
 'the': 2195,
 'non': 1491,
 'cliche': 396,
 'choices': 382,
 'with': 2493,
 'parents': 1584,
 'in': 1122,
 'other': 1548,
 'movies': 1446,
 'could': 484,
 'predict': 1677,
 'dialog': 587,
 'verbatim': 2388,
 'but': 313,
 'writing': 2528,
 'movie': 1445,
 'made': 1343,
 'better': 239,
 'selections': 1913,
 'overall': 1558,
 'rate': 1747,
 '10': 0,
 'out': 1552,
 'of': 1518,
 'scale': 1871,
 'plot': 1644,
 'doesn': 627,
 'hang': 1001,
 'together': 2247,
 'at': 165,
 'all': 88,
 'and': 110,
 'acting': 47,
 'absolutely': 33,
 'appalling': 133,
 'literally': 1299,
 'vomited': 2416,
 'while': 2472,
 'watching': 2444,
 'film': 832,
 'convoluted': 474,
 'just': 1213,
 'didn': 592,
 'convince': 472,
 'me': 1378,
 'much': 1449,
 'was': 2434,
 'watched': 2443,
 'weird': 2459,
 'questioning': 1735,
 'glance': 951,
 'basic': 207,
 'premise': 1682,
 '

In [52]:
from sklearn.datasets import fetch_20newsgroups

In [55]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

In [56]:
twenty_train = fetch_20newsgroups(subset='train',
...     categories=categories, shuffle=True, random_state=42)

In [59]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [64]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [69]:
twenty_train.target_names[twenty_train.target[0]]

'comp.graphics'

In [70]:
# from sklearn.feature_extraction.text import CountVectorizer

In [71]:
count_vect = CountVectorizer()

In [72]:
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [74]:
X_train_counts

<2257x35788 sparse matrix of type '<class 'numpy.int64'>'
	with 365886 stored elements in Compressed Sparse Row format>

In [75]:
count_vect.vocabulary_.get(u'algorithm')

4690

In [141]:
count_vect.vocabulary_

{'from': 14887,
 'sd345': 29022,
 'city': 8696,
 'ac': 4017,
 'uk': 33256,
 'michael': 21661,
 'collier': 9031,
 'subject': 31077,
 'converting': 9805,
 'images': 17366,
 'to': 32493,
 'hp': 16916,
 'laserjet': 19780,
 'iii': 17302,
 'nntp': 23122,
 'posting': 25663,
 'host': 16881,
 'hampton': 16082,
 'organization': 23915,
 'the': 32142,
 'university': 33597,
 'lines': 20253,
 '14': 587,
 'does': 12051,
 'anyone': 5201,
 'know': 19458,
 'of': 23610,
 'good': 15576,
 'way': 34755,
 'standard': 30623,
 'pc': 24651,
 'application': 5285,
 'pd': 24677,
 'utility': 33915,
 'convert': 9801,
 'tif': 32391,
 'img': 17389,
 'tga': 32116,
 'files': 14281,
 'into': 18268,
 'format': 14676,
 'we': 34775,
 'would': 35312,
 'also': 4808,
 'like': 20198,
 'do': 12014,
 'same': 28619,
 'hpgl': 16927,
 'plotter': 25361,
 'please': 25337,
 'email': 12833,
 'any': 5195,
 'response': 27836,
 'is': 18474,
 'this': 32270,
 'correct': 9932,
 'group': 15837,
 'thanks': 32135,
 'in': 17556,
 'advance': 4378,

In [77]:
# from sklearn.feature_extraction.text import TfidfTransformer

In [81]:
# tf
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [83]:
X_train_tf = tf_transformer.transform(X_train_counts)

In [80]:
# tf-idf; Term Frequency times Inverse Document Frequency
tfidf_transformer = TfidfTransformer()

In [84]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [86]:
# from sklearn.naive_bayes import MultinomialNB

In [88]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [89]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']

In [91]:
X_new_counts = count_vect.transform(docs_new)

In [92]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [93]:
predicted = clf.predict(X_new_tfidf)

In [97]:
for doc, category in zip(docs_new, predicted):
    print('%r => %s' %(doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [98]:
# make vectorizer => transformer => classifier easier to work with...
# Pipeline class
# from sklearn.pipeline import Pipeline

In [99]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB()),
])

In [100]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [101]:
twenty_test = fetch_20newsgroups(subset='test',
                          categories=categories, shuffle=True, random_state=42)

In [102]:
docs_test = twenty_test.data

In [104]:
predicted = text_clf.predict(docs_test)

In [105]:
np.mean(predicted == twenty_test.target)

0.8348868175765646

In [106]:
# support vector machine (SVM), regarded as one of best text clf algorithms 
# (but slow)
from sklearn.linear_model import SGDClassifier

In [107]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                         alpha=1e-3, random_state=42,
                                         max_iter=5, tol=None)),
])

In [108]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [110]:
predicted = text_clf.predict(docs_test)

In [111]:
np.mean(predicted==twenty_test.target)

0.9127829560585885

In [112]:
# from sklearn import metrics

In [113]:
print(metrics.classification_report(twenty_test.target, predicted,
                                   target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [114]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]], dtype=int64)

In [ ]:
line_length = []
for l in a:
    line_length.append(len(re.split(', | |-|:|\\n|\\t  ', l)))


In [ ]:
l = []
for i, row in imdb.iterrows():
    if 'good' in row['desc']:
        l.append(1)
    else:
        l.append(0)
        

In [ ]:
imdb['good'] = l

In [ ]:
for i, row in imdb.iterrows():
    print (row)

In [ ]:
string.punctuation

In [ ]:

# words = re.split(r'\W+', a[0])
words = a[788].split()
table = str.maketrans('', '', string.punctuation)
stripped = [w.lower().translate(table) for w in words]
stripped

In [ ]:

for i in a.index:
    print(re.split(', | |-', a[i]))

In [ ]:

count_word = Counter()
punct = ',.!?:;\'"-()*&^%$#@/\\'
for l in a:
    count_word.update(x.strip(punct).lower() for x in re.split(', | |-|:|\\n|\\t', l))

In [ ]:
re.split(', | |-', a[0])[4].rstrip(punct).lower()

In [ ]:

re.split(', | |-|\\n|\\t', '\t1\nthe')

In [ ]:
re.split(', | |-', 'slow-moving')

In [ ]:
count_word

In [ ]:
import operator
sorted_count = sorted(count_word.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
sorted_count

# not needed

In [ ]:
# create dictionary with frequency count
# use regex and Counter method
count_word1 = {}
punct = ',.!?\'"-*&^%$#@'
for i in a:
    for word in i:
        if word not in count_word:
            count_word[word] = 1
        else:
            count_word[word] += 1

In [ ]:
imdb_neg

In [ ]:
imdb_neg.desc[0]

In [ ]:
re.split(', | |-', imdb_neg.desc[0].lower())[:-2]

In [ ]:
b = imdb_neg.desc.apply(lambda x: re.split(', | |-', x.lower())[:-2])
b

In [ ]:
# imdb_neg['pract'] = b